<a href="https://colab.research.google.com/github/rames4498/computervision_practice/blob/main/video_notebooks/00_tensorflow_fundamentals_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!

See full course materials on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/

## Introduction to Tensors

In [12]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.17.1


In [13]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [14]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [15]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [16]:
# Check the dimension of our vector
vector.ndim

1

In [17]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [18]:
matrix.ndim

2

In [19]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [20]:
# What's the number dimensions of another_matrix?
another_matrix.ndim

2

In [21]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3,],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [22]:
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [23]:
# Create the same tesnor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [24]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Let's try change our unchangable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

🔑 **Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers.

In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tesnor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

In [ ]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

In [ ]:
not_shuffled

🛠 **Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [ ]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

### Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10, 7])

In [ ]:
# Create a tensor of all zeroes
tf.zeros(shape=(3, 4))

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

In [ ]:
A = tf.constant(numpy_A, shape=(3, 8))
B = tf.constant(numpy_A)
A, B

In [ ]:
3 * 8

In [ ]:
A.ndim

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

In [ ]:
2 * 3 * 4 * 5

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

### Indexing tensors

Tensors can be indexed just like Python lists.


In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
some_list[:1]

In [ ]:
rank_4_tensor.shape

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

In [ ]:
rank_2_tensor

In [ ]:
some_list, some_list[-1]

In [ ]:
# Get the last item of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

In [ ]:
# Expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0) # expand the 0-axis

In [ ]:
rank_2_tensor

### Manipulating tensors (tensor operations)

**Basic operations**

`+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

In [ ]:
# Original tensor is unchanged
tensor

In [ ]:
# Multiplication also works
tensor * 10

In [ ]:
# Substraction if you want
tensor - 10

In [ ]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

In [ ]:
tensor

**Matrix multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [ ]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

In [ ]:
tensor, tensor

In [ ]:
tensor * tensor

In [ ]:
# Matrix mutliplication with Python operator "@"
tensor @ tensor

In [ ]:
tensor.shape

In [ ]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# Create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

X, Y

In [ ]:
# Try to matrix multiply tensors of same shape
tf.matmul(X, Y)

In [ ]:
Y

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2, 3))

In [ ]:
X.shape, tf.reshape(Y, shape=(2, 3)).shape

In [ ]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
tf.reshape(X, shape=(2, 3)).shape, Y.shape

In [ ]:
# Try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

In [ ]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

📖 **Resource:** Info and example of matrix multiplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html

**The dot product**

Matrix multiplication is also referrred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [ ]:
X, Y

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

In [ ]:
# Perform matrix mutliplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [25]:
# Check the values of Y, reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n") # "\n" is for newline

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:


NameError: name 'Y' is not defined

In [ ]:
tf.matmul(X, tf.transpose(Y))

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will tranpose (rather than reshape) one of the tensors to get satisify the matrix multiplication rules.

### Changing the datatype of a tensor

In [ ]:
tf.__version__

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

In [ ]:
C = tf.constant([7, 10])
C, C.dtype

In [ ]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
D = tf.constant([-7, -10])
D

In [ ]:
# Get the absolute values
tf.abs(D)

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

In [ ]:
tf.size(E), E.shape, E.ndim

In [ ]:
# Find the minimum
tf.reduce_min(E)

In [ ]:
# Find the maximum
tf.reduce_max(E)

In [ ]:
# Find the mean
tf.reduce_mean(E)

In [ ]:
# Find the sum
tf.reduce_sum(E)

🛠 **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods.

In [ ]:
# Find the variance of our tensor
tf.reduce_variance(E) # won't work...

In [ ]:
# To find the variance of our tensor, we need access to tensorflow_probability
import tensorflow_probability as tfp
tfp.stats.variance(E)

In [ ]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))

In [ ]:
tf.math.reduce_std(E)

In [ ]:
# Find the variance of our E tensor
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

### Find the positional maximum and minimum



In [ ]:
# Create a new tensor for finding positional minimum and maxmimum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

In [ ]:
# Find the positional maximum
tf.argmax(F)

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

In [ ]:
# Find the max value of F
tf.reduce_max(F)

In [ ]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

In [ ]:
# Find the positional minimum
tf.argmin(F)

In [ ]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

In [ ]:
G.shape

In [ ]:
# Squeezing removes all single dimensions from a tensor
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="yo I love deep learning", off_value="I also like to dance")

### Squaring, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

In [ ]:
# Square it
tf.square(H)

In [ ]:
# Find the squareroot (will error, method requires non-int type)
tf.sqrt(H)

In [ ]:
# Find the squareroot
tf.sqrt(tf.cast(H, dtype=tf.float32))

In [ ]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays.

🔑 **Note:** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing).

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

In [ ]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

### Finding access to GPUs

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

In [ ]:
!nvidia-smi

> 🔑 **Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenver possible.